# Step 4: Employability and Quality Proxies

## Objective
Add proxies for the "Quality" and "Opportunity" of the destination, which are key pull factors.

## Data Sources
1.  **Youth Transition (TRANS)**: Employment rates for recent graduates (aged 25-29).
2.  **University Rankings**: (Optional) Presence of top-ranked universities.

## Methodology
- **Employment Rate**: We use the employment rate of tertiary graduates as a proxy for the *probability* of finding a job after graduation.
- **Imputation**: Missing years are imputed using nearest-neighbor interpolation (±1 or ±2 years) to maximize coverage.

In [3]:
from pathlib import Path
import re

import numpy as np
import pandas as pd
from IPython.display import display

BASE_DIR = Path("/Users/simonedinato/Documents/Classes/Applied Econometrics/Project")
DATA_DIR = BASE_DIR / "Datasets"

fact_path = DATA_DIR / "07_fact_tables" / "od_fact_table.csv"
trans_path = DATA_DIR / "04_employability_proxy" / "Youth's transition from education to work (TRANS)-2.csv"

# 1) Load inputs
fact = pd.read_csv(fact_path)
trans = pd.read_csv(trans_path, low_memory=False)

# Ensure year is numeric early for downstream joins
fact["year"] = pd.to_numeric(fact["year"], errors="coerce").astype("Int64")

# 2) Schema audit
measure_col = "UNIT_MEASURE" if "UNIT_MEASURE" in trans.columns else "MEASURE"
attainment_col_candidates = ["ATTAINMENT_LEV", "EDUCATION_LEV"]
attainment_col = next((c for c in attainment_col_candidates if c in trans.columns), None)
if attainment_col is None:
    raise ValueError("No tertiary attainment column found in TRANS input")

def log_unique(df: pd.DataFrame, column: str, limit: int = 10) -> None:
    if column not in df.columns:
        print(f"- {column}: column missing")
        return
    uniques = pd.unique(df[column].dropna())
    preview_vals = sorted(str(u) for u in uniques[:limit])
    suffix = " ..." if len(uniques) > limit else ""
    print(f"- {column}: {len(uniques)} unique -> {preview_vals}{suffix}")

print("Schema audit — unique values")
for col in [measure_col, "LABOUR_FORCE_STATUS", "SEX", "AGE", attainment_col]:
    log_unique(trans, col)

measure_values = trans[measure_col].dropna().astype(str)
lf_tokens = sorted({val for val in measure_values if ("LF" in val.upper()) or ("LABOUR" in val.upper())})
pop_tokens = sorted({val for val in measure_values if "POP" in val.upper()})
print("Denominator families detected based on measure tokens:")
print(f"  LF tokens: {lf_tokens if lf_tokens else ['<none>']}")
print(f"  POP tokens: {pop_tokens if pop_tokens else ['<none>']}")

# iso lookup from fact destinations/origins
iso_lookup = pd.concat(
    [
        fact[["destination_country_code", "destination_country"]].rename(
            columns={"destination_country_code": "country_code", "destination_country": "country"}
        ),
        fact[["origin_country_code", "origin_country"]].rename(
            columns={"origin_country_code": "country_code", "origin_country": "country"}
        ),
    ],
    ignore_index=True,
).dropna(subset=["country_code", "country"])
iso_lookup["country"] = iso_lookup["country"].astype(str).str.strip()
iso_lookup["country_code"] = iso_lookup["country_code"].astype(str).str.upper()
iso_lookup = iso_lookup.drop_duplicates(subset=["country"]).set_index("country")["country_code"]

# 3) Filter to target population
attainment_series = trans[attainment_col].astype(str)
tertiary_pattern = r"(ISCED11A_5T8|ISCED11A_5|ISCED11A_6|ISCED11A_7|ISCED11A_8|TERTIARY)"
tertiary_mask = attainment_series.str.contains(tertiary_pattern, case=False, na=False)

age_col = "AGE" if "AGE" in trans.columns else ("Age" if "Age" in trans.columns else None)
if age_col:
    age_series = trans[age_col].astype(str).str.upper()
    age_priority = {"Y25T34", "Y25T29", "Y25T24", "Y25T30", "Y25"}
    age_mask = age_series.isin(age_priority)
    if not age_mask.any():
        age_mask = age_series.str.contains("25", na=False)
else:
    age_mask = True

sex_col = "SEX" if "SEX" in trans.columns else ("Sex" if "Sex" in trans.columns else None)
if sex_col:
    sex_series = trans[sex_col].astype(str).str.upper()
    sex_mask = sex_series.isin({"_T", "T", "TOTAL"})
else:
    sex_mask = True

lf_status_col = "LABOUR_FORCE_STATUS" if "LABOUR_FORCE_STATUS" in trans.columns else None
if lf_status_col:
    lf_status_series = trans[lf_status_col].astype(str).str.upper()
    lf_mask = lf_status_series.isin({"EMP", "UNE"})
else:
    lf_mask = True

edu_status_col = "EDU_STATUS" if "EDU_STATUS" in trans.columns else None
if edu_status_col:
    edu_status_series = trans[edu_status_col].astype(str).str.upper()
    edu_status_mask = edu_status_series.isin({"NED", "NED_NE", "NE", "_T", "TOTAL"})
else:
    edu_status_mask = True

measure_series_upper = trans[measure_col].astype(str).str.upper()
lf_measure_mask = measure_series_upper.str.contains("LF|LABOUR", regex=True, na=False)
pop_measure_mask = measure_series_upper.str.contains("POP", regex=True, na=False)
trans["denom_family"] = np.select(
    [lf_measure_mask, pop_measure_mask],
    ["LF", "POP"],
    default=pd.NA,
)
denom_mask = trans["denom_family"].isin({"LF", "POP"})

combined_mask = tertiary_mask & age_mask & sex_mask & lf_mask & edu_status_mask & denom_mask
filter_cols = [
    "REF_AREA",
    "Reference area",
    "TIME_PERIOD",
    "LABOUR_FORCE_STATUS",
    "denom_family",
    "OBS_VALUE",
]
trans_filtered = trans.loc[combined_mask, filter_cols].copy()

ref_area_clean = trans_filtered["REF_AREA"].astype(str).str.strip()
trans_filtered["country_code"] = np.where(ref_area_clean.str.len() == 3, ref_area_clean.str.upper(), np.nan)
trans_filtered["country_code"] = trans_filtered["country_code"].fillna(
    trans_filtered["Reference area"].astype(str).str.strip().map(iso_lookup)
)
trans_filtered["year"] = pd.to_numeric(trans_filtered["TIME_PERIOD"], errors="coerce").astype("Int64")
trans_filtered["LABOUR_FORCE_STATUS"] = trans_filtered["LABOUR_FORCE_STATUS"].astype(str).str.upper()
trans_filtered["value"] = pd.to_numeric(trans_filtered["OBS_VALUE"], errors="coerce")
over_pct_mask = trans_filtered["value"] > 1.5
trans_filtered.loc[over_pct_mask, "value"] = trans_filtered.loc[over_pct_mask, "value"] / 100.0
trans_filtered["value"] = trans_filtered["value"].clip(lower=0.0, upper=1.0)
trans_filtered = trans_filtered.dropna(subset=["country_code", "year", "value"])
print(f"Filtered TRANS rows: {len(trans_filtered):,}")

# 4) Build tidy matrices per denominator family

def build_family(df: pd.DataFrame, family_label: str) -> pd.DataFrame:
    subset = df[df["denom_family"] == family_label].copy()
    if subset.empty:
        return pd.DataFrame(columns=["country_code", "year"])
    tidy = subset[["country_code", "year", "LABOUR_FORCE_STATUS", "value"]].dropna(
        subset=["country_code", "year", "LABOUR_FORCE_STATUS"]
    )
    if tidy.empty:
        return pd.DataFrame(columns=["country_code", "year"])
    matrix = (
        tidy.pivot_table(
            index=["country_code", "year"],
            columns="LABOUR_FORCE_STATUS",
            values="value",
            aggfunc="median",
        )
        .reset_index()
    )
    matrix.columns.name = None
    return matrix

lf_matrix = build_family(trans_filtered, "LF")
pop_matrix = build_family(trans_filtered, "POP")

if not lf_matrix.empty:
    if "EMP" not in lf_matrix.columns:
        lf_matrix["EMP"] = np.nan
    if "UNE" not in lf_matrix.columns:
        lf_matrix["UNE"] = np.nan
    lf_matrix["emp_lf"] = np.nan
    denom = lf_matrix["EMP"] + lf_matrix["UNE"]
    ratio_mask = lf_matrix["EMP"].notna() & lf_matrix["UNE"].notna() & (denom > 0)
    lf_matrix.loc[ratio_mask, "emp_lf"] = lf_matrix.loc[ratio_mask, "EMP"] / denom[ratio_mask]
    fallback_mask = ~ratio_mask & lf_matrix["EMP"].notna()
    lf_matrix.loc[fallback_mask, "emp_lf"] = lf_matrix.loc[fallback_mask, "EMP"]
    lf_matrix["emp_lf"] = lf_matrix["emp_lf"].clip(lower=0.0, upper=1.0)
    lf_ready = lf_matrix[["country_code", "year", "emp_lf"]].dropna(subset=["emp_lf"])
else:
    lf_ready = pd.DataFrame(columns=["country_code", "year", "emp_lf"])

if not pop_matrix.empty:
    if "EMP" not in pop_matrix.columns:
        pop_matrix["EMP"] = np.nan
    pop_matrix["emp_pop"] = pop_matrix["EMP"].clip(lower=0.0, upper=1.0)
    pop_ready = pop_matrix[["country_code", "year", "emp_pop"]].dropna(subset=["emp_pop"])
else:
    pop_ready = pd.DataFrame(columns=["country_code", "year", "emp_pop"])

# 5-6) Nearest-year smoothing helpers

def smooth_variant(
    source_df: pd.DataFrame,
    value_col: str,
    prefix: str,
    key_pairs: list[tuple[str, int]],
) -> pd.DataFrame:
    expected_cols = ["country_code", "year", value_col]
    if source_df.empty or not all(col in source_df.columns for col in expected_cols):
        return pd.DataFrame(
            columns=["destination_country_code", "year", f"{prefix.lower()}_value", f"{prefix.lower()}_tag", f"{prefix.lower()}_imputed"]
        )
    mapping = (
        source_df.set_index(["country_code", "year"])[value_col]
        .dropna()
        .to_dict()
    )
    if not mapping:
        return pd.DataFrame(
            columns=["destination_country_code", "year", f"{prefix.lower()}_value", f"{prefix.lower()}_tag", f"{prefix.lower()}_imputed"]
        )
    records: list[dict] = []
    for code, year in key_pairs:
        value = None
        tag = None
        imputed = 0
        key = (code, year)
        if key in mapping:
            value = mapping[key]
            tag = f"{prefix}|exact"
        else:
            for gap in (1, 2):
                candidates = []
                for sign in (-gap, gap):
                    candidate_key = (code, year + sign)
                    if candidate_key in mapping:
                        candidates.append(mapping[candidate_key])
                if candidates:
                    value = float(np.mean(candidates))
                    tag = f"{prefix}|±{gap}y"
                    imputed = 1
                    break
        if value is not None:
            value = float(np.clip(value, 0.0, 1.0))
            records.append(
                {
                    "destination_country_code": code,
                    "year": year,
                    f"{prefix.lower()}_value": value,
                    f"{prefix.lower()}_tag": tag,
                    f"{prefix.lower()}_imputed": imputed,
                }
            )
    return pd.DataFrame(records)

dest_keys_df = fact[["destination_country_code", "year"]].copy()
dest_keys_df = dest_keys_df.dropna(subset=["destination_country_code", "year"]).drop_duplicates()
dest_key_pairs = list(dest_keys_df.itertuples(index=False, name=None))

lf_assign = smooth_variant(lf_ready, "emp_lf", "LF", dest_key_pairs)
pop_assign = smooth_variant(pop_ready, "emp_pop", "POP", dest_key_pairs)
if not lf_assign.empty:
    lf_assign["lf_imputed"] = lf_assign["lf_imputed"].astype("Int64")
if not pop_assign.empty:
    pop_assign["pop_imputed"] = pop_assign["pop_imputed"].astype("Int64")

dest_table = dest_keys_df.merge(lf_assign, how="left", on=["destination_country_code", "year"])
dest_table = dest_table.merge(pop_assign, how="left", on=["destination_country_code", "year"])
lf_mask_final = dest_table.get("lf_value").notna() if "lf_value" in dest_table.columns else pd.Series(False, index=dest_table.index)
pop_mask_final = ~lf_mask_final & dest_table.get("pop_value").notna() if "pop_value" in dest_table.columns else pd.Series(False, index=dest_table.index)

dest_table["emp_rate_dest_final"] = dest_table.get("lf_value")
dest_table.loc[pop_mask_final, "emp_rate_dest_final"] = dest_table.loc[pop_mask_final, "pop_value"]

dest_table["emp_source_tag"] = dest_table.get("lf_tag")
dest_table.loc[pop_mask_final, "emp_source_tag"] = dest_table.loc[pop_mask_final, "pop_tag"]

dest_table["emp_rate_imputed"] = pd.NA
if "lf_imputed" in dest_table.columns:
    dest_table.loc[lf_mask_final, "emp_rate_imputed"] = dest_table.loc[lf_mask_final, "lf_imputed"].fillna(0).astype(int)
if "pop_imputed" in dest_table.columns:
    dest_table.loc[pop_mask_final, "emp_rate_imputed"] = dest_table.loc[pop_mask_final, "pop_imputed"].fillna(0).astype(int)
dest_table["emp_rate_imputed"] = dest_table["emp_rate_imputed"].astype("Int64")

dest_table = dest_table[["destination_country_code", "year", "emp_rate_dest_final", "emp_source_tag", "emp_rate_imputed"]]

# 7-8) Merge into fact
cols_to_drop = [
    "emp_rate_dest",
    "emp_rate_dest_missing",
    "emp_rate_imputed",
    "emp_source_tag",
    "emp_logit",
    "emp_z",
]
fact = fact.drop(columns=[col for col in cols_to_drop if col in fact.columns])

fact = fact.merge(dest_table, on=["destination_country_code", "year"], how="left")
fact["emp_rate_dest"] = fact["emp_rate_dest_final"].clip(lower=0.0, upper=1.0)
fact.drop(columns=["emp_rate_dest_final"], inplace=True)
fact["emp_rate_imputed"] = fact["emp_rate_imputed"].where(fact["emp_rate_dest"].notna(), pd.NA).astype("Int64")

emp_rate_numeric = pd.to_numeric(fact["emp_rate_dest"], errors="coerce")
fact["emp_rate_dest"] = emp_rate_numeric
winsorized = emp_rate_numeric.clip(lower=0.02, upper=0.98)
fact["emp_logit"] = np.where(
    emp_rate_numeric.notna(),
    np.log(winsorized / (1 - winsorized)),
    np.nan,
)


def zscore(series: pd.Series) -> pd.Series:
    mean = series.mean()
    std = series.std(ddof=0)
    if pd.isna(std) or std == 0:
        return pd.Series(np.nan, index=series.index)
    return (series - mean) / std

fact["emp_z"] = fact.groupby("year")["emp_rate_dest"].transform(zscore)

# 9) QC checks
coverage = fact["emp_rate_dest"].notna().mean()
emp_stats = fact["emp_rate_dest"].agg(["min", "median", "max"])
source_counts = fact["emp_source_tag"].value_counts(dropna=False)
imputed_total = fact["emp_rate_imputed"].fillna(0).astype(int).sum()

print(f"emp_rate_dest coverage: {coverage:.2%}")
print(f"emp_rate_dest min/median/max: {emp_stats['min']:.3f}, {emp_stats['median']:.3f}, {emp_stats['max']:.3f}")
print("emp_source_tag counts:")
print(source_counts)
print(f"Imputed (±1y/±2y) count: {imputed_total}")

weight_check = (
    fact.groupby(["origin_country_code", "year"])["weight_od"].sum().reset_index(name="w_sum")
)
violations = weight_check[~weight_check["w_sum"].between(0.999, 1.001)]
print("Weight sum violations:", len(violations))
if not violations.empty:
    display(violations.head())

top10 = fact[fact["emp_rate_dest"].notna()].sort_values("emp_rate_dest", ascending=False).head(10)
bottom10 = fact[fact["emp_rate_dest"].notna()].sort_values("emp_rate_dest", ascending=True).head(10)
print("Top 10 destinations by emp_rate_dest:")
display(top10[["destination_country_code", "destination_country", "year", "emp_rate_dest", "emp_source_tag"]])
print("Bottom 10 destinations by emp_rate_dest:")
display(bottom10[["destination_country_code", "destination_country", "year", "emp_rate_dest", "emp_source_tag"]])

# 10) Persist
fact = fact.drop_duplicates()
fact.to_csv(fact_path, index=False)

print(
    {
        "rows": len(fact),
        "coverage_pct": round(coverage * 100, 2),
        "min": round(emp_stats["min"], 3),
        "median": round(emp_stats["median"], 3),
        "max": round(emp_stats["max"], 3),
        "imputed_count": int(imputed_total),
        "weight_violations": int(len(violations)),
    }
)



Schema audit — unique values
- UNIT_MEASURE: 1 unique -> ['PT_POP_SUB']
- LABOUR_FORCE_STATUS: 5 unique -> ['EMP', 'NE', 'OLF', 'POP', 'UNE']
- SEX: 1 unique -> ['_T']
- AGE: 1 unique -> ['Y25T29']
- ATTAINMENT_LEV: 6 unique -> ['ISCED11A_0T2', 'ISCED11A_34_44', 'ISCED11A_35_45', 'ISCED11A_3_4', 'ISCED11A_5T8', '_T']
Denominator families detected based on measure tokens:
  LF tokens: ['<none>']
  POP tokens: ['PT_POP_SUB']
Filtered TRANS rows: 148
emp_rate_dest coverage: 95.83%
emp_rate_dest min/median/max: 0.328, 0.421, 0.917
emp_source_tag counts:
emp_source_tag
POP|exact    621
NaN           27
Name: count, dtype: int64
Imputed (±1y/±2y) count: 0
Weight sum violations: 0
Top 10 destinations by emp_rate_dest:


/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_20887/450240572.py:67: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tertiary_mask = attainment_series.str.contains(tertiary_pattern, case=False, na=False)


,destination_country_code,destination_country,year,emp_rate_dest,emp_source_tag
473,POL,Poland,2022,0.917449,POP|exact
377,POL,Poland,2022,0.917449,POP|exact
257,POL,Poland,2022,0.917449,POP|exact
449,POL,Poland,2022,0.917449,POP|exact
233,POL,Poland,2022,0.917449,POP|exact
425,POL,Poland,2022,0.917449,POP|exact
593,POL,Poland,2022,0.917449,POP|exact
209,POL,Poland,2022,0.917449,POP|exact
113,POL,Poland,2022,0.917449,POP|exact
545,POL,Poland,2022,0.917449,POP|exact


Bottom 10 destinations by emp_rate_dest:


,destination_country_code,destination_country,year,emp_rate_dest,emp_source_tag
529,AUT,Austria,2022,0.327932,POP|exact
361,AUT,Austria,2022,0.327932,POP|exact
73,AUT,Austria,2022,0.327932,POP|exact
97,AUT,Austria,2022,0.327932,POP|exact
121,AUT,Austria,2022,0.327932,POP|exact
217,AUT,Austria,2022,0.327932,POP|exact
457,AUT,Austria,2022,0.327932,POP|exact
49,AUT,Austria,2022,0.327932,POP|exact
25,AUT,Austria,2022,0.327932,POP|exact
385,AUT,Austria,2022,0.327932,POP|exact


{'rows': 648, 'coverage_pct': np.float64(95.83), 'min': np.float64(0.328), 'median': np.float64(0.421), 'max': np.float64(0.917), 'imputed_count': 0, 'weight_violations': 0}
